TensorFlow엔 자동미분 기능이 있음.

<br>

## Gradient Tape

<br>

- 주어진 입력변수에 대한 연산의 그래디언트를 계산하는 것을 위한 **tf.GradientTape API**를 제공하고 있음.
- **context안에서 실행된 모든 연산을 tape에 '기록'** 한다.
- 이후 **revere mode differentiation을 이용해 tape에 기록된 연산의 그래디언트를 계산**한다.
    - 어떻게 보면 역전파를 알아서 계산해준다는 소리인거 같은데

In [5]:
import tensorflow as tf

x = tf.ones((2,2))

# 테이프 기록 과정
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

# 입력 텐서 x에 대한 z의 도함수 계산
dz_dx = t.gradient(z, x)
for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0
print(dz_dx)

# 중간 값 y에 대한 그래디언트 또한 계산이 가능.
# with tf.GradientTape() as t 로 다시 해줘야 함. 안그러면 런타임에러.
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0
print(dz_dy)

tf.Tensor(
[[8. 8.]
 [8. 8.]], shape=(2, 2), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


- 기본적으로 GradientTape.gradient() 가 호출되면 GradientTape에 포함된 리소스가 해제됨.
- 동일한 연산에 대해 여러 그래디언트를 계산하려면 persistent한 테이프를 생성하면 된다.
    - (그래서 아까 에러났던거인듯)

In [8]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x * x
    z = y * y
dz_dx = t.gradient(z, x)
print(dz_dx, '\n')
dy_dx = t.gradient(y, x)
print(dy_dx, '\n')
del t # 더 안쓸거면 메모리 할당 해제해주자. 참조삭제.

tf.Tensor(108.0, shape=(), dtype=float32) 

tf.Tensor(6.0, shape=(), dtype=float32) 



    연산이 실행되는 순서대로 테이프에 기록되므로,
    Python의 제어흐름문 이 자연스럽게 처리 됨.

In [14]:
def f(x,y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            print('결과값에 x를 곱합니다.', f'{output, x.numpy()}')
            output = tf.multiply(output, x)
    return output

def grad(x, y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x, y)
    return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)

print(grad(x, 6).numpy())
print(grad(x, 5).numpy())
print(grad(x, 4).numpy())

결과값에 x를 곱합니다. (1.0, 2.0)
결과값에 x를 곱합니다. (<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, 2.0)
결과값에 x를 곱합니다. (<tf.Tensor: shape=(), dtype=float32, numpy=4.0>, 2.0)
12.0
결과값에 x를 곱합니다. (1.0, 2.0)
결과값에 x를 곱합니다. (<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, 2.0)
결과값에 x를 곱합니다. (<tf.Tensor: shape=(), dtype=float32, numpy=4.0>, 2.0)
12.0
결과값에 x를 곱합니다. (1.0, 2.0)
결과값에 x를 곱합니다. (<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, 2.0)
4.0
